In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic

In [2]:
# Defining classes for the dataframes
class AmenityPoint:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

class AmenityPolygon:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

In [3]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [4]:
# Read the building footprints data

buildingfootprints_gdf = gpd.read_file('manila_building_footprints.geojson')

buildingfootprints_gdf.head()

,id,geometry
0,0,"POLYGON ((120.95954 14.62190, 120.95945 14.621..."
1,1,"POLYGON ((120.96084 14.62342, 120.96095 14.623..."
2,2,"POLYGON ((120.96181 14.60838, 120.96172 14.608..."
3,3,"POLYGON ((120.96087 14.62855, 120.96103 14.628..."
4,4,"POLYGON ((120.96158 14.62105, 120.96162 14.621..."


In [5]:
# Load the Manila amenities data into a Geopandas dataframe
from shapely import wkt

manila_amenities_df = pd.read_csv('manila_amenities.csv')
manila_amenities_df['geometry'] = manila_amenities_df['geometry'].apply(wkt.loads)
manila_amenities_gdf = gpd.GeoDataFrame(manila_amenities_df, crs='epsg:3123')

# Separate into point and polygon dataframes
manila_amenities_polygon_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Polygon']
manila_amenities_point_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Point']
manila_amenities_multipoly_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'MultiPolygon']

# Append multipolygons to the polygon dataframe
manila_amenities_polygon_gdf = gpd.GeoDataFrame(pd.concat([manila_amenities_polygon_gdf, manila_amenities_multipoly_gdf], ignore_index=True))

# Reset point dataframe index
manila_amenities_point_gdf.reset_index(drop=True, inplace=True)

# Add a column to the polygon dataframe to store a list of Amenity Points within the polygon
manila_amenities_polygon_gdf['amenity_points'] = None

In [6]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []

    for j, point in manila_amenities_point_gdf.iterrows():
        try:
            if polygon['geometry'].intersects(point['geometry']):
                # Append the index of the current point
                points_within_polygon.append(j)
        except Exception as e:
            print(f"Error processing polygon {i} point {j}: {e}")
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [7]:
manila_amenities_polygon_gdf['amenity'].unique()

array(['education', 'finance', 'government offices', 'grocery', 'health',
       'malls', 'residential areas', 'security'], dtype=object)

In [8]:
manila_amenities_polygon_gdf[manila_amenities_polygon_gdf['name'].isnull()]

,amenity,addr_city,name,x,y,geometry,amenity_points
1,education,Manila,NaN,120.988929,14.603745,"POLYGON ((120.989 14.604, 120.989 14.604, 120....",[81]
144,education,Manila,NaN,120.992154,14.560229,"POLYGON ((120.992 14.560, 120.992 14.560, 120....",[]
196,education,Manila,NaN,120.979357,14.610687,"POLYGON ((120.980 14.610, 120.979 14.610, 120....",[]
219,education,Manila,NaN,121.008584,14.580380,"POLYGON ((121.009 14.580, 121.009 14.580, 121....",[]
223,education,Manila,NaN,121.008665,14.572657,"POLYGON ((121.009 14.573, 121.009 14.573, 121....",[]
...,...,...,...,...,...,...,...
1227,residential areas,Manila,NaN,120.977209,14.608424,"POLYGON ((120.977 14.609, 120.977 14.609, 120....",[]
1239,security,Manila,NaN,120.982806,14.607571,"POLYGON ((120.983 14.608, 120.983 14.608, 120....",[]
1265,security,Manila,NaN,120.970972,14.602583,"POLYGON ((120.971 14.603, 120.971 14.603, 120....",[]
1266,security,Manila,NaN,120.959548,14.630220,"POLYGON ((120.959 14.630, 120.960 14.630, 120....",[]


In [9]:
# Reading population data
manila_population_df = pd.read_csv('manila-population-polygon.csv')
manila_population_df['geometry'] = manila_population_df['geometry'].apply(wkt.loads)
manila_population_gdf = gpd.GeoDataFrame(manila_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in manila_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('population.html') # Save the map to an HTML file

In [10]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe

# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

   # Add polygon nodes
    for index, row in amenities_polygon_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row and 'amenity_points' in row:
            # Generate a unique node identifier for polygons
            node_id = f"polygon_{index}"
            amenities_network.add_node(node_id, polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], amenity_points=row['amenity_points'])
        else:
            print(f"Skipping row {index} in amenities_polygon_gdf due to missing data.")

    # Add point nodes
    for index, row in amenities_point_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row:
            # Generate a unique node identifier for points
            node_id = f"point_{index}"
            amenities_network.add_node(node_id, point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], is_in_polygon=False)
        else:
            print(f"Skipping row {index} in amenities_point_gdf due to missing data.")
            
    return amenities_network

### For Filtering the Roads

In [225]:
#Plotting filtered roads FOR VISUALIZATION ONLY
def plot_all_filtered_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in filtered_roads_strings.iterrows():
        line_coords = list(road['geometry'].coords)

        if road['highway'] == 'primary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

        if road['highway'] == 'secondary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='red').add_to(m)

        if road['highway'] == 'tertiary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='green').add_to(m)

        if road['highway'] == 'unclassified':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='orange').add_to(m)

    for index, road in filtered_roads_lists.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='purple').add_to(m)

    # Return the map
    return m

def plot_all_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_road.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_private_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_private.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_walk():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_walk.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m
    
# TEMPORARY TO BE REMOVED
def plot_bike():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_bike.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

In [ ]:
# Check which roads can have stops by checking the nearness and intersection of zones
def check_roads_in_zones():

    # Iterate through each zone
    for node_key, node_data in list(graph_networks_of_polygons.nodes.items()):

        # Iterate through each road
        for index, road in filtered_roads_strings.iterrows():
            

        for index, road in filtered_roads_lists.iterrows():
            line = LineString(line_coords)

            zone1_polygon = Polygon(zone1_coords)

            line_intersects_zone1 = line.intersects(zone1_polygon)
            line_intersects_zone2 = line.intersects(zone2_polygon)

            
            
            if line_intersects_zone1:
                print("Line intersects with Zone 1")
            if line_intersects_zone2:
                print("Line intersects with Zone 2")

### For Amenity and zone connection

In [11]:
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
#             contained_points = []
            
#             #Check first if it is already in a list of polygons to be merged
#             for merge_list in list_to_merge:
#                 if node_key in merge_list:
#                     nodes_to_merge = merge_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not nodes_to_merge:
#                 nodes_to_merge.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = float(geodesic(c1, c2).meters)

#                         if distance < max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 nodes_to_merge.append(other_node_key)
                                
#                                 # Checks if contiguous, if not add a line
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             if nodes_to_merge not in list_to_merge:
#                 list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
                
#     temp_graph = to_graph(list_to_merge)
#     lists = graph_to_list(temp_graph)


#     # Now we will finally connect all polygons in the list
#     for merge_list in lists:
#         first = True
#         for node_key in merge_list:
#             if first:
#                 combined_node = graph.nodes[node_key]['geometry']
#                 combined_node_amenity = graph.nodes[node_key]['amenity']
#                 combined_node_key = node_key
#                 combined_node_geometry = graph.nodes[node_key]['geometry']
#                 combined_node_name = graph.nodes[node_key]['name']
#                 combined_node_lat = graph.nodes[node_key]['lat']
#                 combined_node_lon = graph.nodes[node_key]['lon']
#                 combined_node_points = graph.nodes[node_key]['amenity_points']
#                 first = False
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                 combined_node_geometry = combined_node
#                 combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                 combined_node_lat = combined_node_geometry.centroid.x
#                 combined_node_lon = combined_node_geometry.centroid.x
#                 combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
#         combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                 lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)

In [12]:
# # V2
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
#             contained_points = []
            
#             #Check first if it is already in a list of polygons to be merged
#             for merge_list in list_to_merge:
#                 if node_key in merge_list:
#                     nodes_to_merge = merge_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not nodes_to_merge:
#                 nodes_to_merge.append(node_key)
            
#             # Distance 
#             total_distance = 0 # This is to calculate the total distance
#             combined_node = graph.nodes[node_key]['geometry']
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
                
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
#                         # Create polygon buffer
#                         poly1 = node_data['geometry'].buffer(meters_to_degrees(30))
#                         poly2 = other_node_data['geometry'].buffer(meters_to_degrees(30))

#                         # Calculate the intersection of the buffered polygons
#                         intersection = poly1.intersection(poly2)

#                         # Check if they intersect
#                         if intersection.area > 0:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
                            
#                             # Check if it does not exceed the max perimeter
#                             combined_node = shapely.ops.unary_union([combined_node, graph.nodes[other_node_key]['geometry']])
#                             total_distance += degrees_to_meters(combined_node.length)
                            
#                             if not amenities_intersecting and total_distance < max_perimeter:
#                                 nodes_to_merge.append(other_node_key)
                                
#                                 # Checks if contiguous, if not add a line - FOR VISUALIZATION
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             if nodes_to_merge not in list_to_merge:
#                 list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
                
#     temp_graph = to_graph(list_to_merge)
#     lists = graph_to_list(temp_graph)


#     # Now we will finally connect all polygons in the list
#     for merge_list in lists:
#         first = True
#         for node_key in merge_list:
#             if first:
#                 combined_node = graph.nodes[node_key]['geometry']
#                 combined_node_amenity = graph.nodes[node_key]['amenity']
#                 combined_node_key = node_key
#                 combined_node_geometry = graph.nodes[node_key]['geometry']
#                 combined_node_name = graph.nodes[node_key]['name']
#                 combined_node_lat = graph.nodes[node_key]['lat']
#                 combined_node_lon = graph.nodes[node_key]['lon']
#                 combined_node_points = graph.nodes[node_key]['amenity_points']
#                 first = False
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                 combined_node_geometry = combined_node
#                 combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                 combined_node_lat = combined_node_geometry.centroid.x
#                 combined_node_lon = combined_node_geometry.centroid.x
#                 combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
#         combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                 lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)

In [164]:
# V3
# Lines version - Non-Merging
# Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            nodes_to_merge = []
            contained_points = []
            
            # Add this node to the list as default
            nodes_to_merge.append((node_key, 0))
            
            for other_node_key, other_node_data in list(graph.nodes.items()):
                
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                    if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
                        distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                        # Check if they are within distance of each other
                        if distance <= max_distance:

                            # Check if any other amenities intersect the line between centroids
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
                            print("Polygons: ", node_data['name'], ", ", other_node_data['name']) 
                            # If no other amenities intersecting and if there is no big road in the middle
                            if not amenities_intersecting and not find_intersecting_roads(line_between_centroids):    
                                nodes_to_merge.append((other_node_key, distance))
                                
                                # Checks if contiguous, if not add a line - FOR VISUALIZATION
                                if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
                                    connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
                elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
                    #Add the point to the new graph if it is not there yet
                    if other_node_key not in combined_graph.nodes:
                        combined_graph.add_node(other_node_key, **other_node_data)
                        
                    if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
                        contained_points.append(other_node_key)
            
            nodes_to_merge.sort(key=lambda x: x[1])
            list_to_merge.append(nodes_to_merge) # Add to the list to connect the polygons later

            for point_key in contained_points:
                graph.nodes[point_key]['is_in_polygon'] = True
    
    
    # Now we will finally connect all polygons in the list
    nodes_added = [] # TO make sure there are no duplicates
    for merge_list in list_to_merge:
        first = True
        added = False
        for node_key, distance in merge_list:
            if node_key not in nodes_added:
                if first:
                    combined_node_amenity = graph.nodes[node_key]['amenity']
                    combined_node_key = node_key
                    combined_node_geometry = graph.nodes[node_key]['geometry']
                    combined_node_name = graph.nodes[node_key]['name']
                    combined_node_lat = graph.nodes[node_key]['lat']
                    combined_node_lon = graph.nodes[node_key]['lon']
                    combined_node_points = graph.nodes[node_key]['amenity_points']
                    nodes_added.append(node_key)
                    first = False
                    added = True
                else:
                    combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                    if degrees_to_meters(combined_node.length) < max_perimeter:
                        combined_node_geometry = combined_node
                        combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                        combined_node_lat = combined_node_geometry.centroid.x
                        combined_node_lon = combined_node_geometry.centroid.x
                        combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                        nodes_added.append(node_key)
                    else:
                        break
                        
        if added:
            combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                    lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None
        
#FUNCTION FOR VISUALIZATION
def connect_polygon_lines(p1, p2):
    centroid1 = p1.centroid
    centroid2 = p2.centroid

    # Create a LineString from the centroids
    line_between_polygons = LineString([centroid1, centroid2])
    connected_lines.append(line_between_polygons)


def find_intersecting_roads(line):
    for index, row in filtered_roads_strings.iterrows():
        if line.intersects(row['geometry']):
            if row['highway'] in separation_options:
                print("Road found: ", row['name'])
                return True

    for index, row in filtered_roads_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                print("Road found: ", row['name'])
                return True
                
    print("NO BIG ROADS")
    return False

In [14]:
# To merge duplicates in lists
def to_graph(nodes):
    G = nx.Graph()
    for part in nodes:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(nodes):
    it = iter(nodes)
    last = next(it)

    for current in it:
        yield last, current
        last = current
        
def graph_to_list(G):
    connected_components = nx.connected_components(G)
    lists = [list(component) for component in connected_components]
    return lists

In [15]:
def plot_network_on_map(amenities_network, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = amenity_colors[data.get('amenity')]
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [16]:
# # Function to connect zones in a network
# def create_zone_network(graph, max_distance):
#     connect_graph = nx.Graph()
#     network_id = 1
#     list_to_connect = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             connect_nodes = []
            
#             #Check first if it is already in a list of polygons to be connected
#             for connect_list in list_to_connect:
#                 if node_key in connect_list:
#                     connect_nodes = connect_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not connect_nodes:
#                 connect_nodes.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     if node_key != other_node_key:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = geodesic(c1, c2).meters

#                         if distance < max_distance: # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 connect_nodes.append(other_node_key)
                                
#             if connect_nodes not in list_to_connect:
#                 list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
#     temp_graph = to_graph(list_to_connect)
#     lists = graph_to_list(temp_graph)

                
#     # Connect them to one network
#     for network in lists:
#         for node_key in network:
#             if node_key not in connect_graph.nodes:
#                 node_attributes = graph.nodes[node_key]
#                 node_attributes["network_id"] = network_id
#                 connect_graph.add_node(node_key, **node_attributes)
#         network_id += 1
                
#     return connect_graph

In [172]:
# V2
# Function to connect zones in a network
def create_zone_network(graph, max_distance):
    connect_graph = nx.Graph()
    network_id = 1
    list_to_connect = []
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            connect_nodes = []
            
            #Check first if it is already in a list of polygons to be connected
            for connect_list in list_to_connect:
                if node_key in connect_list:
                    connect_nodes = connect_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not connect_nodes:
                connect_nodes.append(node_key)
                
            # If this is not a res
            if node_key not in pop_graph or not pop_graph.nodes[node_key]['is_a_zone']:
                for other_node_key, other_node_data in list(graph.nodes.items()):
                    if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                        if node_key != other_node_key:

                            distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                            # Check if they are within distance of each other
                            if distance <= max_distance:
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
                                if not amenities_intersecting and not find_intersecting_roads(line_between_centroids):
                                    if other_node_key not in pop_graph or not pop_graph.nodes[other_node_key]['is_a_zone']:
                                        connect_nodes.append(other_node_key)
                                
            if connect_nodes not in list_to_connect:
                list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
    temp_graph = to_graph(list_to_connect)
    lists = graph_to_list(temp_graph)

                
    # Connect them to one network
    for network in lists:
        for node_key in network:
            if node_key not in connect_graph.nodes:
                node_attributes = graph.nodes[node_key]
                node_attributes["network_id"] = network_id
                connect_graph.add_node(node_key, **node_attributes)
        network_id += 1
                
    return connect_graph

In [18]:
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [19]:
# Uses the combined graph
# Formula: Population Density = Total Population / Total Area

def check_residential_population_density(graph, threshold):
    # Create an empty array to store points
    points = []
    
    pop_graph = nx.Graph()
    for index, row in manila_population_gdf.iterrows():
        # Append the point to the points array
        points.append((row['latitude'], row['longitude'], row['phl_general_2020']))
    
    
    nodes = []
    for node_key, node_data in list(graph.nodes.items()):
        # Check if its a polygon and is a residential area
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            total_pop = 0
            
            for point in points:
                if Point(point[1], point[0]).within(node_data['geometry']):
                    total_pop += point[2] # Add the density
                    points.remove(point)
            
            density = total_pop / node_data['geometry'].area
            
            if (density > 0):
                node_data["is_a_zone"] = True
            else:
                node_data["is_a_zone"] = False
                
            nodes.append((node_key, density))
            pop_graph.add_node(node_key, **node_data)
                
    return pop_graph

In [20]:
# This is to better visualize which residential areas can become zones
def plot_population_zones_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    
                    if (data['is_a_zone']):
                        folium.Polygon(locations=coordinates, fill=True, color="green", fill_opacity=0.4).add_to(m)
                    else:
                        folium.Polygon(locations=coordinates, fill=True, color="red", fill_opacity=0.4).add_to(m)

    # Return the map
    return m

In [21]:
# Extract the graph into a geojson for loading into QGIS

def graph_to_geojson(graph, filename):
    # Initialize an empty list to hold GeoJSON features
    features = []

    # Iterate over the nodes in the graph
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Convert the geometry to a GeoJSON-compatible format
            geometry = shapely.geometry.shape(data['geometry'])
            # Create a copy of the properties to check for NaN values
            properties = data.copy()
            # Remove the geometry from the properties
            properties.pop('geometry', None)
            # Check for NaN values in the properties
            if all(not (isinstance(value, float) and np.isnan(value)) for value in properties.values()):
                # Create a GeoJSON feature for the node
                feature = geojson.Feature(geometry=geometry, properties=properties)
                # Add the feature to the list
                features.append(feature)

    # Create a GeoJSON FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Return the GeoJSON FeatureCollection
    return feature_collection

## Main

In [22]:
# Creating Initial network
manila_amenities_network = create_network(manila_amenities_polygon_gdf, manila_amenities_point_gdf)

# Make a before map
before_map = plot_network_on_map(manila_amenities_network, initial_location=[0, 0], zoom_start=100)
before_map.save('before_map.html') # Save the map to an HTML file

In [175]:
# Considering the road type
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    place = 'Manila, Philippines'
    mode = 'drive'
    graph = ox.graph_from_place(place, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, 'map/Manila.graphml') # Save it as a file

def load_graph():
    graph = ox.load_graphml('map/Manila.graphml')
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph



#generate_graph()
graph = load_graph()

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(graph,nodes=False, edges=True)

# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose widths are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Get all the roads with set widths (Widths that are not null)
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926




In [170]:
# Connecting polygons of same amenity
connected_lines = []
combined_graph = combine_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save('after_map.html') # Save the map to an HTML file

Polygons:  nan ,  Nihongo Center
Polygons:  nan ,  University of the East
Road found:  Loyola Street
Polygons:  nan ,  Mary Chile's College
Road found:  Loyola Street
Polygons:  nan ,  FEU Gymnasium
Polygons:  nan ,  FEU Institute of Technology Building
NO BIG ROADS
Polygons:  nan ,  Philippine College of Business Administration
NO BIG ROADS
Polygons:  nan ,  University of Manila Basic Education Department
Polygons:  Nihongo Center ,  nan
Polygons:  Nihongo Center ,  Far Eastern University
Road found:  Nicanor Reyes Street
Polygons:  Nihongo Center ,  University of the East
Road found:  Loyola Street
Polygons:  Nihongo Center ,  FEU Gymnasium
NO BIG ROADS
Polygons:  Nihongo Center ,  FEU Institute of Technology Building
Polygons:  Nihongo Center ,  Philippine College of Business Administration
NO BIG ROADS
Polygons:  Lyceum of the Philippines University ,  Mapúa University
NO BIG ROADS
Polygons:  Doña Teodora Alonzo High School ,  Francisco Balagtas Elementary School
NO BIG ROADS
Polyg

In [25]:
# Checking the population density of residential areas
pop_graph = check_residential_population_density(graph=combined_graph, threshold=100)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)
#pop_map.save('pop_map.html') # Save the map to an HTML file

In [173]:
# Create a network of amenities
graph_networks_of_polygons = create_zone_network(graph=combined_graph, max_distance=100)
networks_map = plot_connected_zones_network_on_map(graph_networks_of_polygons, initial_location=[0, 0], zoom_start=100)
networks_map.save('networks_map.html') # Save the map to an HTML file

feature_collection = graph_to_geojson(manila_amenities_network, 'output.geojson')
with open('output.geojson', 'w', encoding='utf-8') as f:
    f.write(geojson.dumps(feature_collection, indent=2))

NO BIG ROADS
Road found:  Padre Faura Street
Road found:  Padre Faura Street
Road found:  Maria Orosa Street
NO BIG ROADS
NO BIG ROADS
Road found:  Loyola Street
Road found:  Loyola Street
NO BIG ROADS
NO BIG ROADS
Road found:  Nicanor Reyes Street
NO BIG ROADS
NO BIG ROADS
Road found:  Victoria Street
NO BIG ROADS
Road found:  Quiricada Street
NO BIG ROADS
Road found:  Abad Santos Avenue
Road found:  Masangkay Street
NO BIG ROADS
Road found:  Abad Santos Avenue
NO BIG ROADS
Road found:  Masangkay Street
Road found:  C. M. Recto Avenue
NO BIG ROADS
NO BIG ROADS
NO BIG ROADS
NO BIG ROADS
Road found:  Padre Faura Street
NO BIG ROADS
Road found:  Pureza Street
NO BIG ROADS
NO BIG ROADS
NO BIG ROADS
NO BIG ROADS
Road found:  Magsaysay Boulevard
Road found:  C. M. Recto Avenue
Road found:  F. R. Hidalgo Street
Road found:  San Rafael Street
Road found:  Legarda Street
Road found:  P. Casal Street
Road found:  C. M. Recto Avenue
NO BIG ROADS
Road found:  Bilibid Viejo Street
Road found:  Bil

In [178]:
# FOR VISUALIZATION ONLY
all_roads_map = plot_all_roads()
all_roads_map.save('all_roads.html')

filtered_road_map = plot_all_filtered_roads()
filtered_road_map.save('filtered_road_map.html')

In [ ]:
# Get the zones that are close to those roads
graph_network_of_polygons
# Step 5: Check intersection/nearness


# Add zones to the map for visualization
folium.Polygon(locations=[(y, x) for x, y in zone1_coords], color='green', fill=True, fill_color='green', fill_opacity=0.4).add_to(mymap)
folium.Polygon(locations=[(y, x) for x, y in zone2_coords], color='orange', fill=True, fill_color='orange', fill_opacity=0.4).add_to(mymap)

# Save or display the map
mymap.save("map_with_line.html")



# Map the zones

In [125]:
highways = rows_with_strings['highway'].values

unique_types = []

for x in highways:
    if x not in unique_types:
        unique_types.append(x)

unique_types

['residential',
 'secondary',
 'trunk',
 'primary',
 'tertiary',
 'secondary_link',
 'unclassified',
 'primary_link',
 'trunk_link',
 'motorway_link',
 'motorway',
 'busway',
 'living_street',
 'tertiary_link',
 'emergency_bay',
 'escape']

In [176]:
manila_road.loc[manila_road['name'] == 'E. Mendiola Street']

,,,osmid,oneway,lanes,name,highway,maxspeed,reversed,length,ref,geometry,width,access,bridge,junction,tunnel
u,v,key,,,,,,,,,,,,,,,
29487757,1037131083,0,901861603,True,1,E. Mendiola Street,tertiary,40,False,98.385,NaN,"LINESTRING (120.99303 14.59830, 120.99368 14.5...",NaN,permissive,NaN,NaN,NaN
68080230,29487757,0,"[5019457, 26600283, 126350685]",True,"[3, 2]",E. Mendiola Street,tertiary,40,False,315.885,NaN,"LINESTRING (120.99096 14.60031, 120.99108 14.6...",NaN,NaN,yes,NaN,NaN
68082882,33453037,0,"[89416992, 1077721624, 28788650]",True,3,E. Mendiola Street,tertiary,40,False,323.061,NaN,"LINESTRING (120.99317 14.59833, 120.99308 14.5...",NaN,NaN,yes,NaN,NaN
1037130787,68082882,0,901861604,True,1,E. Mendiola Street,tertiary,40,False,90.120,NaN,"LINESTRING (120.99377 14.59776, 120.99356 14.5...",NaN,permissive,NaN,NaN,NaN


In [130]:
filtered_roads_lists

,,,osmid,name,highway,oneway,reversed,length,lanes,maxspeed,width,geometry,ref,junction,access,tunnel,bridge,service
u,v,key,,,,,,,,,,,,,,,,
25389142,2669725315,0,"[4245395, 918503355, 866465126, 866465125]",Taft Avenue,"[tertiary, secondary]",True,False,280.880,"[2, 1]",40,NaN,"LINESTRING (120.99722 14.53275, 120.99752 14.5...",NaN,NaN,NaN,building_passage,NaN,NaN
29670528,27499236,0,"[42059267, 857094727, 857094728, 846260180, 44...",Pacific Avenue,"[motorway_link, secondary]",True,False,541.606,"[2, 1]","[60, 40]","[4, 5]","LINESTRING (120.98767 14.50587, 120.98975 14.5...",NaN,NaN,NaN,NaN,yes,NaN
33443923,33445248,0,"[161306512, 1076793612, 784669680]",Lawton Road,"[tertiary, secondary_link]",True,False,246.699,2,40,NaN,"LINESTRING (120.98012 14.59534, 120.97996 14.5...",NaN,NaN,NaN,NaN,NaN,NaN
56349606,11438211667,0,"[664348318, 1232882054]","[Champaca Street, Rosal Street]","[tertiary, residential]",False,"[False, True]",79.128,2,30,NaN,"LINESTRING (121.01211 14.40111, 121.01204 14.4...",NaN,NaN,destination,NaN,NaN,NaN
56349607,11438211667,0,"[575789645, 1232882053]","[Champaca Street, Rosal Street]","[tertiary, residential]",False,"[False, True]",161.018,2,30,NaN,"LINESTRING (121.01186 14.39903, 121.01189 14.3...",NaN,NaN,destination,NaN,NaN,NaN
95516147,321306174,0,"[824904122, 824904123]",Doña Julia Vargas Avenue,"[secondary_link, secondary]",True,False,239.797,1,40,NaN,"LINESTRING (121.05795 14.58436, 121.05943 14.5...",NaN,NaN,NaN,NaN,NaN,NaN
243461262,243433378,0,"[762325832, 22696913]","[Captain M. Reyes Street, Arguelles Street]","[tertiary, residential]",True,False,120.496,2,"[30, 20]",NaN,"LINESTRING (121.01058 14.54946, 121.01038 14.5...",NaN,NaN,NaN,NaN,NaN,NaN
243844405,3074652157,0,"[28740361, 918548714, 919438858, 22732213, 227...",NaN,"[primary_link, tertiary]",True,False,503.841,"[3, 2]","[40, 20]",NaN,"LINESTRING (121.00305 14.51226, 121.00298 14.5...",NaN,NaN,NaN,NaN,yes,NaN
299691817,320184590,0,"[958786896, 29108941]",Colonel Miguel Nuke Street,"[tertiary, residential]",False,"[False, True]",42.963,2,NaN,NaN,"LINESTRING (121.05350 14.76768, 121.05346 14.7...",NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
len(manila_road)

12617